In [ ]:
import numpy as np
from sympy import symbols, init_printing, gcd, expand
from sympy.polys.domains import GF,ZZ,QQ

from collections import defaultdict


def fullchar(wght, G):
    '''
    This function returns the full character of the repn of 'G' with highest weight 'wght'.
    Input:
        wght = tuple of integers (the highest weight of the representation)
        G = string e.g. 'A2', 'G2', 'E8' (the group name)
    '''
    return lie.W_orbit(lie.dom_char(wght, G), G)


def coeffs(p):
    '''
    Takes a polynomial (of type LiE element?) and returns its coefficients as a list of integers
    '''
    length = int(lie.length(p))
    return [int(lie.coef(p, i + 1)) for i in range(length)]


def exponents(p):
    '''
    Takes a polynomial  (of type LiE element?) and returns its exponents as a list of tuples of integers
    '''
    length = int(lie.length(p))
    return [list(map(int, lie.expon(p, i + 1))) for i in range(length)]


class LengthError(Exception):
    """Exception for when input length is not the right size."""
    pass


if __name__ == "__main__":
    G = input("Enter group name as 'Capital letter + number', e.g. A2, A4, F4, G2, E6, ...")
    print
    rankG = int(lie.Lie_rank(G))  # rank of the group

    # Produce a list of lists of fundamental weights, there will be rank(G) of these.
    fund_weights_list = [[1 if i == j else 0 for j in range(rankG)] for i in range(rankG)]

    fund_characters = []
    fund_polys_coeff_list = []
    fund_polys_exponent_list = []
    for weight in fund_weights_list:
        p = fullchar(weight, G)
        fund_characters.append(p)
        fund_polys_coeff_list.append(coeffs(p))
        fund_polys_exponent_list.append(exponents(p))

    go_again = 'y'
    while go_again != 'n':
        while True:
            try:
                Rval = tuple(int(item) for item in input("Enter the value of H at each simple root, order them as in Humphrey's, each separated by a space: ").split())
                if len(Rval) != rankG:
                    raise LengthError
                break
            except LengthError:
                print(f"You have input the wrong number of integers. Please insert {rankG} integers, each separated by space.")
            except ValueError:
                print("One or more entries were not of type int. Please try again.")

        while True:
            #Input the characteristic of prime field they wish to calculate K over.
            try:
                char = int(input("Enter characteristic of prime field you want K calculated over: "))
                break
            except ValueError:
                print("Please insert a prime or 0.")

        # We now have a list of integers telling us how H acts on simple roots. However we need to know how H acts on the fundamental weights.
        # To make this conversion we use the Cartan matrix, or rather its inverse, since (weight values)^T = C^{-1}(root values)^T
        C_hold = lie.Cartan(G)
        C = [[int(C_hold[i][j]) for j in range(1, rankG+1)] for i in range(1, rankG+1)]
        Cinv = np.linalg.inv(C)

        # A list of values of H on the fundamental weights, this is simply C^{-1}(value of H on roots)^T
        Wval_hold = Cinv.dot(np.array(Rval))

        # Make an list with the same data as Wval_hold, but with floats instead of numpy objects
        Wval = list(map(float, Wval_hold))

        # Restrict the character of each fundamental representation to SL2. Here we use 'Wval' which tells us how H acts on the weights.
        res_poly_master_list = []
        for k, p in enumerate(fund_characters):
            restriction_exponents = [int(round(sum(Wval[j] * fund_polys_exponent_list[k][i][j] for j in range(rankG)))) for i in range(int(lie.length(p)))]

            # Pair the restriction_exponents and fund_polys_coeff_list by index and sort by exponents in descending order.
            coefficients_by_exponents = defaultdict(int)
            for exponent, coefficient in zip(restriction_exponents, fund_polys_coeff_list[k]):
                coefficients_by_exponents[exponent] += coefficient

            min_exponent = min(restriction_exponents)
            max_exponent = max(restriction_exponents)
            sorted_exponents = list(range(max_exponent, min_exponent - 1, -1))
            sorted_coefficients = [coefficients_by_exponents[exp] for exp in sorted_exponents]

            res_poly_master_list.append([sorted_coefficients, sorted_exponents])

        #Note: symbols and init_printing are from the sympy package
        x = symbols('x')
        init_printing(use_unicode=False, wrap_line=False)

        sl2_irrep_polys = [1, x]  # e.g. where P_0 = 1, P_1 = x and use [P_1][P_n] = [P_{n-1}]+[P_{n+1}] to calculate further
        max_irrep = max(poly[1][0] for poly in res_poly_master_list)  #maximum of the restricted exponents from above a.k.a the maximum P_i that we will need.

        for i in range(2, max_irrep + 1):
            X = sl2_irrep_polys[1] * sl2_irrep_polys[i - 1] - sl2_irrep_polys[i - 2]
            sl2_irrep_polys.append(expand(X))

        # Initialize char_polys_wrt_P_i_coeffs with zeros. We will write each restricted character as a sum of the P_i (i.e. as a sum of irreducibles).
        char_polys_wrt_P_i_coeffs = [[0] * (max_irrep + 1) for _ in range(rankG)]

        # Decompose the restricted characters into sum of the P_i (sum of sl2 irreps)
        for i in range(rankG):
            poly_coeffs, exponents = res_poly_master_list[i]
            for j in range(len(poly_coeffs)):
                coeff = poly_coeffs[j]
                if coeff != 0:
                    char_polys_wrt_P_i_coeffs[i][exponents[j]] += coeff
                    for k in range(exponents[j] + 1):
                        poly_coeffs[j + 2 * k] -= coeff

        # Compute the restricted characters
        char_polys_wrt_x = [sum(char_polys_wrt_P_i_coeffs[i][j] * sl2_irrep_polys[j] for j in range(max_irrep + 1)) for i in range(rankG)]

        # Compute the gcd of above 'character polynomials with respect to x' - 'dimension of character'
        Y = char_polys_wrt_x[0] - int(lie.dim(fund_weights_list[0], G))
        if rankG > 1:
            for i in range(1, rankG):
                if char == 0:
                    Y = gcd(char_polys_wrt_x[i] - int(lie.dim(fund_weights_list[i], G)), Y, domain=QQ)
                elif char > 0:
                    Y = gcd(char_polys_wrt_x[i] - int(lie.dim(fund_weights_list[i], G)), Y, domain=GF(char))

        # Result
        print("")
        print(f"K^{{0,0}}(G/SL2) \\otimes F when H acts as {Rval} on the simple roots is...")
        print("")
        print(f"F[x]/({Y})")
        print("")
        print("Would you like to input another weighted Dynkin diagram? (type anything for yes / 'n' for no)")
        go_again = input()

Enter group name as 'Capital letter + number', e.g. A2, A4, F4, G2, E6, ... A2
Enter the value of H at each simple root, order them as in Humphrey's, each separated by a space:  2 2
Enter characteristic of prime field you want K calculated over:  0



K^{0,0}(G/SL2) \otimes F when H acts as (2, 2) on the simple roots is...

F[x]/(x**2 - 4)

Would you like to input another weighted Dynkin diagram? (type anything for yes / 'n' for no)


 y
